# Example: using OpenMM to run a simulation with NequIP ML Potential

You can run this tutorial directly in your browser: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openmm/openmm-torch/blob/master/tutorials/openmm-torch-nnpops.ipynb)


## Install Conda

[Conda](https://docs.conda.io/) is a package and environment manager. On Google Colab, Conda is installed with [conda-colab](https://github.com/jaimergp/condacolab). On your computer, you should follow these [installation instructions](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).

⚠️ Do not use conda-colab on your computer!

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

## Install software

The [conda-forge](https://conda-forge.org/) channel is used for software.

⚠️ The installation might take up to 10 min!

In [ ]:
#https://github.com/openmm/openmm-torch/issues/88
%env CONDA_OVERRIDE_CUDA=11.2 
!mamba install -c conda-forge openmm-torch pytorch=1.12

!pip install -q torch-nl

!pip install -q git+https://github.com/sef43/openmm-ml@nequip

## Run simulation

In [ ]:
import openmm
import openmm.app as app
import openmm.unit as unit
from openmmml import MLPotential
from sys import stdout

"""
Uses a deployed trained NequiP model, toluene example:
nequip-train configs/example.yaml
nequip-deploy build --train-dir path/to/training/session/ deployed_model.pth
"""

# load toluene structure
pdb = app.PDBFile("toluene.pdb")


# create a System with NequIP MLP
potential = MLPotential('nequip', model_path="deployed_model.pth", atom_type_to_atomic_number={"H":1, "C":6})
system = potential.createSystem(pdb.topology)


# run langevin dynamics at 300K for 200 steps
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 10.0/unit.picoseconds, 1.0*unit.femtosecond)
simulation=app.Simulation(pdb.topology, system, integrator, platform=openmm.Platform.getPlatformByName("CPU"))
simulation.context.setPositions(pdb.positions)
simulation.reporters.append(app.PDBReporter('output.pdb', 10))
simulation.reporters.append(app.StateDataReporter(stdout, 10, step=True,
        potentialEnergy=True, temperature=True))

simulation.step(200)

# Minimize the energy
simulation.minimizeEnergy()
energy=simulation.context.getState(getEnergy=True).getPotentialEnergy()
print(energy, energy.in_units_of(unit.kilocalorie_per_mole))